In [127]:
import os
# import qiime2
import pandas as pd
import logging
import argparse

In [119]:

def import_fq(fq_list, sample_name, tmpdir='./'):
    raw_seqs = None
    try:
        from qiime2 import Artifact
        tmpmanifest = os.path.join(tmpdir, 'tmpmanifest')
        # with open(tmpmanifest, 'w') as h:
        if len(fq_list) == 2:
            df = pd.DataFrame([sample_name] + fq_list).T
            df.columns=['sample-id','forward-absolute-filepath','reverse-absolute-filepath']
            df.to_csv(tmpmanifest, index=False, sep='\t')
            raw_seqs = Artifact.import_data('SampleData[PairedEndSequencesWithQuality]', tmpmanifest, 'PairedEndFastqManifestPhred33V2')
        if len(fq_list) == 1:
            df = pd.DataFrame([sample_name]  + fq_list).T
            df.columns=['sample-id','absolute-filepath']
            df.to_csv(tmpmanifest, index=False, sep='\t')
            raw_seqs = Artifact.import_data('SampleData[SequencesWithQuality]', tmpmanifest, 'SingleEndFastqManifestPhred33V2')
    except Exception as e:
        logging.error(e)
    return raw_seqs

def get_rep_seq(raw_seqs):
    table, rep_seqs, stats = None, None, None
    try:
        import qiime2.plugins.dada2.actions as dada2_actions
        table, rep_seqs, stats = dada2_actions.denoise_single(
            demultiplexed_seqs=raw_seqs, 
            trunc_len=0
        )
    except Exception as e:
        logging.error(e)
    return table, rep_seqs, stats

def get_tax(rep_seq, ref):
    taxonomy = None
    try:
        from qiime2 import Artifact
        import qiime2.plugins.feature_classifier.actions as feature_classifier_actions
        arti_ref = Artifact.load(ref)
        taxonomy, = feature_classifier_actions.classify_sklearn(
            classifier=arti_ref,
            reads=rep_seq,
        )
    except Exception as e:
        logging.error(e)
    return taxonomy

def data2df(data):
    from qiime2 import Metadata
    return data.view(Metadata).to_dataframe()

In [121]:
def fq2asv(fq_list):
    raw_seqs = import_fq(fq, sn)
    table, rep_seqs, stats = get_rep_seq(raw_seqs)
    tax = get_tax(rep_seqs, ref)
    return data2df(stats), pd.concat([data2df(table).T, data2df(rep_seqs), data2df(tax)], axis=1)

In [ ]:
if __name__ == '__main__':
    parse = argparse.ArgumentParser(formatter_class=argparse.ArgumentDefaultsHelpFormatter)
    
    
    args = parse.parse_args()
    
    logging.basicConfig()
    
    stats, tax = fq2asv(fq)
    

In [122]:

# fq = ['/mnt/d/Yangk/work/qiime/test_data/SRR18505770_1.fastq', '/mnt/d/Yangk/work/qiime/test_data/SRR18505770_2.fastq']
# fq1 = ['/mnt/d/Yangk/work/qiime/test_data/SRR18505775_1.fastq', '/mnt/d/Yangk/work/qiime/test_data/SRR18505775_2.fastq']

# sn = 'testName'
# ref = '/mnt/d/Yangk/work/qiime/gg-13-8-99-515-806-nb-classifier.qza'
# stats, tax = fq2asv(fq)

Running external command line application(s). This may print messages to stdout and/or stderr.
The command(s) being run are below. These commands cannot be manually re-run as they will depend on temporary files that no longer exist.

Command: run_dada_single.R /tmp/q2-SingleLanePerSampleSingleEndFastqDirFmt-gvua1bi4 /tmp/tmpsgww050b/output.tsv.biom /tmp/tmpsgww050b/track.tsv /tmp/tmpsgww050b 0 0 2.0 2 Inf independent consensus 1.0 1 1000000 NULL 16

R version 4.1.2 (2021-11-01) 


载入需要的程辑包：Rcpp


DADA2: 1.22.0 / Rcpp: 1.0.8.3 / RcppParallel: 5.1.5 
1) Filtering .
2) Learning Error Rates
3218919 total bases in 16699 reads from 1 samples will be used for learning the error rates.
3) Denoise samples .
4) Remove chimeras (method = consensus)
5) Report read numbers through the pipeline
6) Write output
